### Climate forcing and climate feedbacks

As we discussed in the first lecture radiative forcing is the change in net top-of-atmosphere radiation $N$ caused by a change in state or composition $x_i$. Assuming small perturbations and hence linearity: 

$$
F_i = \frac{d N}{d x_i}
$$

Climate feedbacks are changes in $N$ caused, directly or indirectly, by changes in surface temperature $T_s$

$$
\lambda_i = \frac{\partial N}{\partial x_i} \frac{d  x_i}{d T_s}
$$

In a closed system, such as the global mean or a single column, $N = 0$ in long-term equilibrium (steady state) since otherwise the temperature would change. For a system subject to radiative forcing the net top-of-atmosphere radiation is a balance between the applied forcing and the changing temperature: 
$$
N = F - \lambda \Delta T_s
$$
with 
$$
\lambda = \frac{\partial N}{\partial T_s} + \sum_i  \frac{\partial N}{\partial x_i}\frac{dx_i}{d T_s}
$$

### Identifying feedbacks in RCE

What are the state variables $x_i$ in our one-dimensional model for RCE? As we've seen the system comprises a small number of variables: surface temperature $T_s$ and profiles of temperature $T(p)$ and humidity $q(p)$ (where $q$ implies an absolute concentration of water vapor rather than relative humidity). $N$ depends on the vertical distribution of $T$ and $Q$, each of which are defined on the pressure (vertical) grid, so the full "state vector" includes every individual value of $T$ and $q$. 

It's possible to compute feedbacks with respect to the change of each variable at each level but it's not very enlightening since those values depend so strongly on each other. In this lab we'll decompose feedbacks into changes due to 
1. changes in surface temperature (the surface temperature feedback $\lambda_{T_s}$
2. vertically uniform warming of the atmosphere, or the atmospheric Planck feedback $\lambda_{T}$
3. vertically non-uniform warming of the atmosphere, or the lapse rate feedback $\lambda_{\Gamma}$
4. changes in humidity, or the water vapor feedback $\lambda_{q}$

As you'll see, the latter two are quite closely tied together. 

### Computing feedbacks in RCE 

For all the questions below please start with order 10 combinations of surface temperatures (say in the range 288-310 K) and humidities run to radiative-convective equilibrium with fixed surfacet temperatures, as you did in Question 4, part 1 of Lab 1. 

_Technical advice:_ 

1. Remember that $N$ is available directly from a simulation `rce` as `rce.radiation["toa"][-1]`. 
2. For computing feedbacks, i.e. changes in $N$ with respect to state, use `rce.radiation.update_heatingrates()` as in Question 2 of Lab 1. 
3. Remember that instances of konrad (`rce` in the examples above) are persistent, so if surface temperature is changed for one calculation it remains at that value unless explicitly changed again. 

We will use `konrad`'s default distribution of humidity: constant relative humidity RH of 80% in the tropopshere and constant above the coldest temperature. We developed ways to change the tropospheric RH in Lab 1, question 1. 

Please plot the following quantities in some way that illustrates the interesting dependencies. One possibility is to make plots as a function of $T_s$ and $RH$. 

In [1]:
# Import needed Python modules 
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc

import numpy as np

import konrad

In [2]:
#
# Used in examples below
#
atmosphere = konrad.atmosphere.Atmosphere(konrad.utils.get_quadratic_pgrid(1000e2, 10, 128))
atmosphere["CO2"][:] = 280E-6
initial_conds = atmosphere.copy()

### Surface temperature feedbacks 

In Lab 1 you learned how to update the radiation fields after changing the atmospheric conditions using `konrad.radiation.update_heatingrates()`. First determine the atmospheric state in equilibrium, and save the top-of-atmosphere radiation imbalance; then change the surface temperature by 1 K and compute a new equilibrium

In [3]:
Ts = 294

base = konrad.RCE(atmosphere = initial_conds.copy(), 
                 surface=konrad.surface.FixedTemperature(temperature=Ts))
base.run()
Neq = base.radiation["toa"][-1]

pert = konrad.RCE(atmosphere = base.atmosphere.copy(), 
                 surface=konrad.surface.FixedTemperature(temperature=Ts+1))
pert.run()

The surface temperature feedback is $\frac{dN}{d T_s}$: 

In [4]:
rad = copy.deepcopy(base) 
rad.surface["temperature"][:] = Ts + 1 # Increase the surface temperature
rad.radiation.update_heatingrates(rad.atmosphere, rad.surface)
print ("Surface feedback: ", np.round(rad.radiation["toa"][-1] - Neq, 2, ), " W/m2-K")

Surface feedback:  -0.83  W/m2-K


- For several humidity/surface temperature combinations, compute the surface temperature feedback $\lambda_{T_s} = \frac{\partial N}{\partial T_s}$ by changing surface temperature and computing the change in top-of-atmosphere net radiation $N$ with respect to the equilibrated solution. (You may well have done this in Lab 1, question 4.1). 

### Water vapor and lapse rate feedbacks 

**Revisit** The Planck feedback, i.e. the change in $N$ due to vertically uniform warming, is conceptually simple, but the atmosphere is unlikely to warm uniformly. 

In the atmospheric sciences the term "lapse rate" describes the rate of change of temperature with  or pressure $\Gamma(p) = \frac{dT}{dp}$. In RCE this is determined by the "moist adiabatic lapse rate" $\Gamma_m(p)$ meaning the variation of temperature with height in a energy-conserving parcel of air in which water is condensing. $\Gamma_m(p)$ depends on temperature and pressure through the dependence of the maximum (saturation) water vapor mixing ratio on temperature. 

The "lapse rate feedback" $\lambda_{\Gamma}$ is the change in $N$ due to changes in $\Gamma(p)$ - in other words, the change in $N$ due to non-uniform warming of the atmosphere. 

The water vapor feedback $\lambda_{wv}$, the change in $N$ caused by changes in the distribution of water vapor, is so strongly controlled by temperature that it's easiest to consider $\lambda_{wv}$ and $\lambda_{\Gamma}$ together. 

As discussed by [Isaac Held and Karen Shell (2012)](https://10.1175/JCLI-D-11-00721.1) (an excerpt of which is [below](#held-shell)), the magnitude of the lapse rate and water feedbacks depend on which state variable is used to represent water vapor, which is equivalent to choosing a reference response. If we characterize water vapor by its _absolute concentration_, feedbacks are measured against a world in which absolute concentration doesn't change with temperature, meaning that relative humidity decreases with warming. If we characterize water vapor by its _relative concentration_ or _relative humidity_ (relative being measured with respect to temperature-depednent saturation), then the feedbacks are measured in a world in which warmer is also wetter. 

In our models of RCE, in which relative humidity is constant by construction, using an absolute humidity framwork leads to large lapse rate and water vapor feedbacks of opposite sign, while using a relative humidity framework yields much smaller feedbacks. 

### Planck, lapse rate, and water vapor and lapse rate feedbacks in konrad

Feedbacks can computed by evaluating the $\frac{\partial N}{\partial x_i}$ and $\frac{dx_i}{d T_s}$ terms separately. It's somewhat easier to compute the product directly by substituting different state variables from a baseline and a perturbed equilibrium

In [6]:
# Make a mask for tropospheric values
trop = base.atmosphere["plev"] > 230.*100.

### Feedbacks under fixed Q

### Feedbacks with constant RH

### Excursus: On clarity and choosing a decomposition 
<a id='held-shell'></a>

The following discussion is from from [Held and Shell 2012](https://10.1175/JCLI-D-11-00721.1), arguing for using relative humidity rather than absolute humdity as the state variable when computing climate feedbacks. 

"Consider a system in which net radiation at TOA depends on two state variables $A, B$. Equilibrium is reached after a forcing $\tilde F$ when 
$$ F = -\frac{\partial N}{\partial A}\delta A - \frac{\partial N}{\partial B}\delta B$$

If one variable is the most interesting - say surface temperature - we might rewrite this as 
$$ F = -\frac{\partial N}{\partial A}\delta A - \frac{\partial N}{\partial B}\frac{\delta B}{\delta A}\delta A$$
or 
$$ \delta A = -\frac{F}{\lambda_A + \lambda_B}$$
where 
$$\lambda_A = \frac{\partial N}{\partial A}, \lambda_B = \frac{\partial N}{\partial B}\frac{\delta B}{\delta A}$$

where $\lambda_B$ is the "B feedback"

We could write this in terms of the "reference response" of $A$ to $F$: 

$$\delta A = \frac{\delta A\big|_{B}}{1 - \mu_B}$$

with $\mu_B = -\lambda_B/\lambda_A$  a nondimensional measure of the amplitude of the B feedback, defined so as to be positive if it increases the amplitude of $\delta A$.

Now suppose that $B = f(A,C)$ and we want to think of the feedbacks in terms of $C$ rather than $B$. $B$ might be tropospheric temperature, say, and if $C = A - C$ then $C$ is the lapse rate. Then 
$$ F = -\left(\frac{\partial N}{\partial A} + \frac{\partial N}{\partial B}\frac{\partial B}{\partial A}\right) \delta A - 
\frac{\partial N}{\partial B}\frac{\partial B}{\partial C}\delta C$$
Then 
$$ \delta A = -\frac{F}{\tilde \lambda_A + \tilde \lambda_C} = \frac{\delta A\big|_{C}}{1 - \tilde \mu_C}$$

$$ \tilde \lambda_A = \frac{\partial N}{\partial A} + \frac{\partial N}{\partial B}\frac{\partial B}{\partial A}$$

$$ \tilde \lambda_C = \frac{\partial N}{\partial B}\frac{\partial B}{\partial C}\frac{\delta C}{\delta A}; \tilde \mu_C = -\tilde \lambda_C / \tilde \lambda_A$$

Of course, the system does not care whether we think of the flux as a function of $(A, B)$ or $(A, C)$. The choice between the two is a convention.

Consider a model in which $A$ is the surface temperature, $B$ is the temperature of the troposphere, and $C = A - B$. Then the $C$ feedback would be a lapse-rate feedback, and the reference response at fixed $C$ would be the familiar reference assuming identical temperature perturbations at the surface and through the entire troposphere. The $B$ feedback perspective would be very different. A climate change with uniform warming at the surface and through the troposphere would be described as resulting from a very large reference sensitivity (computed as if the forcing were required to be balanced entirely by the surface warming) in conjunction with a very strong negative $B$, or tropospheric temperature, feedback. 

It is worth considering why the latter, untraditional, viewpoint seems so awkward. Fundamentally, the problem is that it is physically implausible for the surface warming and tropospheric warming to vary independently. The large changes in gravitational stability that would result if there were no "tropospheric feedback" would be strongly resisted by the atmospheric circulation. (RP: This argument leads to RCE.) It makes little sense to use variables $A$ and $B$ in this kind of analysis if they are so closely tied together that the limit of no $B$ feedback is so implausible.

While there is nothing preventing one from using this perspective, it can cause confusion. For example, it might encourage the (incorrect) idea that because $B$ feedback is large its strength must be a source of substantial uncertainty in the response. Thinking of the "tropospheric feedback" as large and negative in this example is simply a result of our odd choice of variables.

It is undesirable for the realizability of a large positive feedback to be dependent on the presence of a large negative feedback $\dots$ A feedback analysis that involves this cancellation makes the decomposition of the response into parts due to different processes look more complicated than it actually is."